In [ ]:
import sys

import pandas as pd
import numpy as np

df_case = pd.read_csv('../datas/covid_worldwide.csv')
df_wid = pd.read_csv('../datas/외교부_국가(지역)별 일반현황_20220629.csv', encoding='cp949')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

In [36]:
df_case.head()

,Serial Number,Country,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Test,Population
0,1,USA,"104,196,861","1,132,935","101,322,779","1,741,147","1,159,832,679","334,805,269"
1,2,India,"44,682,784","530,740","44,150,289","1,755","915,265,788","1,406,631,776"
2,3,France,"39,524,311","164,233","39,264,546","95,532","271,490,188","65,584,518"
3,4,Germany,"37,779,833","165,711","37,398,100","216,022","122,332,384","83,883,596"
4,5,Brazil,"36,824,580","697,074","35,919,372","208,134","63,776,166","215,353,593"


In [ ]:
df_world_map=pd.read_excel('../datas/worldcities.xlsx',engine='openpyxl')
df_world_map=df_world_map[['country','lat','lng']]
df_world_map=df_world_map.drop_duplicates(['country'])
df_world_map=df_world_map.reset_index(drop=True)
df_world_map=df_world_map.sort_values('country')
df_world_map=df_world_map.rename(columns={'country':'국가명', 'lat' : '위도', 'lng': '경도'})
df_world_map=df_world_map.sort_values('국가명')
list_m=df_world_map['국가명'].unique() #국가명이 동일하지않은부분을 찾기위해서 리스트비교분석

In [34]:
df_world_map.head()

,국가명,위도,경도
41,Afghanistan,34.5328,69.1658
138,Albania,41.3300,19.8200
44,Algeria,36.7764,3.0586
202,American Samoa,-14.2740,-170.7046
193,Andorra,42.5000,1.5000


In [4]:
dfc0 = df_case.rename(columns={'Serial Number': '고유번호', 'Country' : '국가명', 'Total Cases' : '확진수',
                        'Total Deaths' : '사망수', 'Total Recovered' : '회복수',
                        'Active Cases': '현발병', 'Total Test': '검사수', 'Population' : '인구'})
print(dfc0.columns)
dfc0.drop(['고유번호','현발병'], axis=1, inplace=True)
dfc0 = dfc0.replace(',','',regex=True) 
dfc0 = dfc0.set_index(['국가명'])
dfc=dfc0.dropna()
for i in dfc.columns:
    dfc[i]=dfc[i].astype('int') #int타입변경
dfc.drop(dfc[dfc['인구']  < (dfc['인구'].mean()*0.05)].index, inplace=True)

Index(['고유번호', '국가명', '확진수', '사망수', '회복수', '현발병', '검사수', '인구'], dtype='object')


C:\Users\hunmi\AppData\Local\Temp\ipykernel_30744\239621498.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc[i]=dfc[i].astype('int')
C:\Users\hunmi\AppData\Local\Temp\ipykernel_30744\239621498.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.drop(dfc[dfc['인구']  < (dfc['인구'].mean()*0.05)].index, inplace=True)


In [43]:
dfc0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231 entries, USA to Tokelau
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   확진수     231 non-null    object
 1   사망수     225 non-null    object
 2   회복수     210 non-null    object
 3   검사수     213 non-null    object
 4   인구      228 non-null    object
dtypes: object(5)
memory usage: 10.8+ KB


In [ ]:
dfc_fin = dfc.copy()
dfc_fin
df_wid = pd.read_csv('../datas/외교부_국가(지역)별 일반현황_20220629.csv', encoding='cp949')
dw1= df_wid[['영문 국가명','면적']]
dw1=dw1.rename(columns={'영문 국가명' : '국가명'})
dw1
dfc_fin=dfc_fin.reset_index()
dfc_fin
C = {'UAE': 83600 , 'Hong Kong': 1114, 'UK':243610 , 'Czechia':78867, 'Italy':302072, 'S. Korea': 100210,
     'USA': 9830000,'Papua New Guinea': 452860, 'CAR' : 623000, 'DRC':2345408, 'Burundi' :  27834}

In [45]:
dw1

,국가명,면적
0,Ghana,238533
1,Gabon,267000
2,Guyana,214969
3,Gambia,11295
4,Guatemala,108889
5,Vatican,0
6,Grenada,348
7,Greece,131957
8,Guinea,246000
9,Guinea-Bissau,36125


In [7]:
dft = pd.merge(dfc_fin,df_world_map,how='outer',on='국가명')
dft = dft.set_index(['국가명'])
dft = pd.merge(dft,dw1, how ='outer', on = '국가명')
dft = dft.set_index(['국가명'])
dft

,확진수,사망수,회복수,검사수,인구,위도,경도,면적
국가명,,,,,,,,
USA,104196861.0,1132935.0,101322779.0,1.159833e+09,3.348053e+08,NaN,NaN,NaN
India,44682784.0,530740.0,44150289.0,9.152658e+08,1.406632e+09,28.6667,77.2167,3287782.0
France,39524311.0,164233.0,39264546.0,2.714902e+08,6.558452e+07,48.8566,2.3522,675417.0
Germany,37779833.0,165711.0,37398100.0,1.223324e+08,8.388360e+07,52.5167,13.3833,357580.0
Brazil,36824580.0,697074.0,35919372.0,6.377617e+07,2.153536e+08,-23.5504,-46.6339,8510000.0
Japan,32588442.0,68399.0,21567425.0,9.214464e+07,1.255848e+08,35.6839,139.7744,378000.0
S. Korea,30197066.0,33486.0,29740877.0,1.580406e+07,5.132990e+07,NaN,NaN,NaN
Italy,25453789.0,186833.0,25014986.0,2.654782e+08,6.026277e+07,41.8931,12.4828,NaN
UK,24274361.0,204171.0,24020088.0,5.225265e+08,6.849791e+07,NaN,NaN,NaN


In [8]:
for a in dft.index:
    if a in C:
        dft.loc[a]['면적'] = C[a]

In [9]:
dict_loc = {'UAE':[23.7139,54.3035], 'CAR': [6.5741,20.4869], 'DRC' : [-2.6046, 22.2650], 'Congo':[-2.9244, 23.7446],
            'Gambia' :[13.4363, -15.4027],'North Macedonia' : [41.6505, 21.6511], 'USA' :[39.5265, -102.0569],
            'UK' :[54.9817, -2.8012], 'S. Korea':[37.5666, 126.9780]}

for nat in dft.index:
    if nat in dict_loc:
        dft.loc[nat]['위도'] = dict_loc[nat][0]
        dft.loc[nat]['경도'] = dict_loc[nat][1]
        
dft

,확진수,사망수,회복수,검사수,인구,위도,경도,면적
국가명,,,,,,,,
USA,104196861.0,1132935.0,101322779.0,1.159833e+09,3.348053e+08,39.5265,-102.0569,9830000.0
India,44682784.0,530740.0,44150289.0,9.152658e+08,1.406632e+09,28.6667,77.2167,3287782.0
France,39524311.0,164233.0,39264546.0,2.714902e+08,6.558452e+07,48.8566,2.3522,675417.0
Germany,37779833.0,165711.0,37398100.0,1.223324e+08,8.388360e+07,52.5167,13.3833,357580.0
Brazil,36824580.0,697074.0,35919372.0,6.377617e+07,2.153536e+08,-23.5504,-46.6339,8510000.0
Japan,32588442.0,68399.0,21567425.0,9.214464e+07,1.255848e+08,35.6839,139.7744,378000.0
S. Korea,30197066.0,33486.0,29740877.0,1.580406e+07,5.132990e+07,37.5666,126.9780,100210.0
Italy,25453789.0,186833.0,25014986.0,2.654782e+08,6.026277e+07,41.8931,12.4828,302072.0
UK,24274361.0,204171.0,24020088.0,5.225265e+08,6.849791e+07,54.9817,-2.8012,243610.0


In [10]:
df_tot = dft.dropna()

In [11]:
df_tot

,확진수,사망수,회복수,검사수,인구,위도,경도,면적
국가명,,,,,,,,
USA,104196861.0,1132935.0,101322779.0,1.159833e+09,3.348053e+08,39.5265,-102.0569,9830000.0
India,44682784.0,530740.0,44150289.0,9.152658e+08,1.406632e+09,28.6667,77.2167,3287782.0
France,39524311.0,164233.0,39264546.0,2.714902e+08,6.558452e+07,48.8566,2.3522,675417.0
Germany,37779833.0,165711.0,37398100.0,1.223324e+08,8.388360e+07,52.5167,13.3833,357580.0
Brazil,36824580.0,697074.0,35919372.0,6.377617e+07,2.153536e+08,-23.5504,-46.6339,8510000.0
Japan,32588442.0,68399.0,21567425.0,9.214464e+07,1.255848e+08,35.6839,139.7744,378000.0
S. Korea,30197066.0,33486.0,29740877.0,1.580406e+07,5.132990e+07,37.5666,126.9780,100210.0
Italy,25453789.0,186833.0,25014986.0,2.654782e+08,6.026277e+07,41.8931,12.4828,302072.0
UK,24274361.0,204171.0,24020088.0,5.225265e+08,6.849791e+07,54.9817,-2.8012,243610.0


In [12]:
# 확진률 (확진수/검사수)
# 사망률 (사망수/확진수)
# 회복률 (회복수/확진수)
# 검사율 (검사수/인구)
col_dict = {'확진수':'검사수', '사망수':'확진수', '회복수': '확진수', '검사수':'인구'}
for j in col_dict:
    j = str(j)
    T = []
    for i in df_tot.index:
        i = str(i)
        T.append(df_tot.loc[i,j]/df_tot.loc[i,str(col_dict[j])]*100)
    df_tot[f'{j}율']= T

C:\Users\hunmi\AppData\Local\Temp\ipykernel_30744\2520404067.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tot[f'{j}율(%)']= T
C:\Users\hunmi\AppData\Local\Temp\ipykernel_30744\2520404067.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tot[f'{j}율(%)']= T
C:\Users\hunmi\AppData\Local\Temp\ipykernel_30744\2520404067.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [13]:
df_tot = df_tot.sort_values(by='검사수율(%)', ascending=False)
df_tot

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율(%),사망수율(%),회복수율(%),검사수율(%)
국가명,,,,,,,,,,,,
Austria,5780229.0,21689.0,5730189.0,2.112735e+08,9.066710e+06,48.2083,16.3725,83879.0,2.735898,0.375227,99.134290,2330.211554
Denmark,3173247.0,8145.0,3163849.0,1.291448e+08,5.834950e+06,55.6805,12.5615,42934.0,2.457124,0.256677,99.703836,2213.296669
UAE,1049409.0,2348.0,1032735.0,1.986388e+08,1.008178e+07,23.7139,54.3035,83600.0,0.528300,0.223745,98.411106,1970.274173
Spain,13731478.0,118434.0,13557699.0,4.710363e+08,4.671914e+07,40.4167,-3.7167,505370.0,2.915163,0.862500,98.734448,1008.229834
Hong Kong,2877280.0,13358.0,2509483.0,7.612387e+07,7.604299e+06,22.3069,114.1831,1114.0,3.779734,0.464258,87.217198,1001.063609
Greece,5708301.0,35630.0,5662212.0,1.022284e+08,1.031664e+07,37.9842,23.7281,131957.0,5.583872,0.624179,99.192597,990.907841
UK,24274361.0,204171.0,24020088.0,5.225265e+08,6.849791e+07,54.9817,-2.8012,243610.0,4.645575,0.841097,98.952504,762.835682
Bahrain,700452.0,1543.0,698400.0,1.068244e+07,1.783983e+06,26.2250,50.5775,765.0,6.557039,0.220286,99.707046,598.797410
Czechia,4590019.0,42312.0,4538304.0,5.689322e+07,1.073678e+07,50.0833,14.4167,78867.0,8.067778,0.921826,98.873316,529.890729


In [14]:
def pop_den(df):  # 인구밀도 열 추가
    T =[]
    for i in df.index:
        T.append(df.loc[i]['인구']/df.loc[i]['면적'])
    df['인구밀도'] = T

In [15]:
pop_den(df_tot)
df_tot

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율(%),사망수율(%),회복수율(%),검사수율(%),인구밀도
국가명,,,,,,,,,,,,,
Austria,5780229.0,21689.0,5730189.0,2.112735e+08,9.066710e+06,48.2083,16.3725,83879.0,2.735898,0.375227,99.134290,2330.211554,108.092729
Denmark,3173247.0,8145.0,3163849.0,1.291448e+08,5.834950e+06,55.6805,12.5615,42934.0,2.457124,0.256677,99.703836,2213.296669,135.905110
UAE,1049409.0,2348.0,1032735.0,1.986388e+08,1.008178e+07,23.7139,54.3035,83600.0,0.528300,0.223745,98.411106,1970.274173,120.595514
Spain,13731478.0,118434.0,13557699.0,4.710363e+08,4.671914e+07,40.4167,-3.7167,505370.0,2.915163,0.862500,98.734448,1008.229834,92.445420
Hong Kong,2877280.0,13358.0,2509483.0,7.612387e+07,7.604299e+06,22.3069,114.1831,1114.0,3.779734,0.464258,87.217198,1001.063609,6826.121185
Greece,5708301.0,35630.0,5662212.0,1.022284e+08,1.031664e+07,37.9842,23.7281,131957.0,5.583872,0.624179,99.192597,990.907841,78.181809
UK,24274361.0,204171.0,24020088.0,5.225265e+08,6.849791e+07,54.9817,-2.8012,243610.0,4.645575,0.841097,98.952504,762.835682,281.178552
Bahrain,700452.0,1543.0,698400.0,1.068244e+07,1.783983e+06,26.2250,50.5775,765.0,6.557039,0.220286,99.707046,598.797410,2332.003922
Czechia,4590019.0,42312.0,4538304.0,5.689322e+07,1.073678e+07,50.0833,14.4167,78867.0,8.067778,0.921826,98.873316,529.890729,136.137852


In [16]:
df_tot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132 entries, Austria to Algeria
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   확진수      132 non-null    float64
 1   사망수      132 non-null    float64
 2   회복수      132 non-null    float64
 3   검사수      132 non-null    float64
 4   인구       132 non-null    float64
 5   위도       132 non-null    float64
 6   경도       132 non-null    float64
 7   면적       132 non-null    float64
 8   확진수율(%)  132 non-null    float64
 9   사망수율(%)  132 non-null    float64
 10  회복수율(%)  132 non-null    float64
 11  검사수율(%)  132 non-null    float64
 12  인구밀도     132 non-null    float64
dtypes: float64(13)
memory usage: 18.5+ KB


In [46]:
df_tot.loc['총합'] = df_tot.iloc[:130,:5].sum()
df_tot.loc['평균'] = df_tot.iloc[:130,:].mean()
df_tot.tail()

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율(%),사망수율(%),회복수율(%),검사수율(%),인구밀도
국가명,,,,,,,,,,,,,
Syria,5.745300e+04,3.164000e+03,5.428900e+04,1.462690e+05,1.936481e+07,36.200000,37.150000,1.851800e+05,39.279000,5.507110,94.492890,0.755334,104.572897
Eritrea,1.018900e+04,1.030000e+02,1.008600e+04,2.369300e+04,3.662244e+06,15.333300,38.916700,1.176000e+05,43.004263,1.010894,98.989106,0.646953,31.141531
Algeria,2.713780e+05,6.881000e+03,1.827490e+05,2.308610e+05,4.535015e+07,36.776400,3.058600,2.381741e+06,117.550387,2.535578,67.341126,0.509063,19.040755
총합,6.379665e+08,6.505524e+06,6.127991e+08,6.499282e+09,5.977695e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
평균,4.907435e+06,5.004249e+04,4.713839e+06,4.999447e+07,4.598227e+07,20.219645,21.569302,9.485729e+05,14.987128,1.624449,95.245127,179.104284,262.816037


In [33]:
df_tot.loc['S. Korea']

확진수        3.019707e+07
사망수        3.348600e+04
회복수        2.974088e+07
검사수        1.580406e+07
인구         5.132990e+07
위도         3.756660e+01
경도         1.269780e+02
면적         1.002100e+05
확진수율(%)    1.910715e+02
사망수율(%)    1.108916e-01
회복수율(%)    9.848929e+01
검사수율(%)    3.078920e+01
인구밀도       5.122233e+02
Name: S. Korea, dtype: float64

In [19]:
df_data=df_tot.copy()
df_data.drop(['총합','평균'],axis=0,inplace=True)

In [20]:
df_data

,확진수,사망수,회복수,검사수,인구,위도,경도,면적,확진수율(%),사망수율(%),회복수율(%),검사수율(%),인구밀도
국가명,,,,,,,,,,,,,
Austria,5780229.0,21689.0,5730189.0,2.112735e+08,9.066710e+06,48.2083,16.3725,83879.0,2.735898,0.375227,99.134290,2330.211554,108.092729
Denmark,3173247.0,8145.0,3163849.0,1.291448e+08,5.834950e+06,55.6805,12.5615,42934.0,2.457124,0.256677,99.703836,2213.296669,135.905110
UAE,1049409.0,2348.0,1032735.0,1.986388e+08,1.008178e+07,23.7139,54.3035,83600.0,0.528300,0.223745,98.411106,1970.274173,120.595514
Spain,13731478.0,118434.0,13557699.0,4.710363e+08,4.671914e+07,40.4167,-3.7167,505370.0,2.915163,0.862500,98.734448,1008.229834,92.445420
Hong Kong,2877280.0,13358.0,2509483.0,7.612387e+07,7.604299e+06,22.3069,114.1831,1114.0,3.779734,0.464258,87.217198,1001.063609,6826.121185
Greece,5708301.0,35630.0,5662212.0,1.022284e+08,1.031664e+07,37.9842,23.7281,131957.0,5.583872,0.624179,99.192597,990.907841,78.181809
UK,24274361.0,204171.0,24020088.0,5.225265e+08,6.849791e+07,54.9817,-2.8012,243610.0,4.645575,0.841097,98.952504,762.835682,281.178552
Bahrain,700452.0,1543.0,698400.0,1.068244e+07,1.783983e+06,26.2250,50.5775,765.0,6.557039,0.220286,99.707046,598.797410,2332.003922
Czechia,4590019.0,42312.0,4538304.0,5.689322e+07,1.073678e+07,50.0833,14.4167,78867.0,8.067778,0.921826,98.873316,529.890729,136.137852
